<a href="https://colab.research.google.com/github/Rorodino/homework-5-Rorodino/blob/main/Copy_of_project5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Question 1
import torch
import torch.nn as nn
import pandas as pd
import statsmodels.api as sm

# Load OASIS dataset
dat = pd.read_csv("https://raw.githubusercontent.com/bcaffo/ds4bme_intro/master/data/oasis.csv")

# Prepare predictor (t2) and outcome (lesion)
X = torch.tensor(dat['T2'].values, dtype=torch.float32).unsqueeze(1)  # shape (n_samples,1)
y = torch.tensor(dat['GOLD_Lesions'].values, dtype=torch.float32).unsqueeze(1)

# Define logistic regression model in PyTorch
class LogisticRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)  # input=1, output=1 (with bias)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return self.sigmoid(self.linear(x))

# Instantiate model, loss, optimizer
model = LogisticRegressionModel()
criterion = nn.BCELoss()  # Binary cross entropy
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

# Train model
for epoch in range(5000):
    optimizer.zero_grad()
    outputs = model(X)
    loss = criterion(outputs, y)
    loss.backward()
    optimizer.step()

# Get PyTorch fitted values
fitted_values_torch = model(X).detach().numpy()

# Fit logistic regression using statsmodels for comparison
X_sm = sm.add_constant(dat['T2'])  # add intercept
logit_model = sm.Logit(dat['GOLD_Lesions'], X_sm).fit(disp=0)
fitted_values_sm = logit_model.predict()

# Compare first 10 fitted values
print("Fitted values (PyTorch):", fitted_values_torch[:10].flatten())
print("Fitted values (Logistic Regression):", fitted_values_sm[:10])


Fitted values (PyTorch): [0.5105911  0.47408372 0.42447558 0.40399653 0.49067637 0.45189273
 0.4430629  0.47003114 0.41406193 0.47743514]
Fitted values (Logistic Regression): [0.57861341 0.52891862 0.45993587 0.43122061 0.55165151 0.49821114
 0.48591648 0.52333433 0.44534063 0.53352775]


In [ ]:
#Question 2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# Load diamond dataset
# Replace with actual path or previous dataset
# Example from seaborn (for demonstration)
import seaborn as sns
df = sns.load_dataset('diamonds')  # this has carat, depth, table, x, y, z, price, etc.

# Select predictors and target
X = df[['carat', 'depth', 'table', 'x', 'y', 'z']].values
y = df['price'].values

# Split into train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = torch.tensor(scaler.fit_transform(X_train), dtype=torch.float32)
X_test = torch.tensor(scaler.transform(X_test), dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
y_test = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

# Define neural network with 2 hidden layers
class DiamondNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(6, 16)   # first hidden layer
        self.fc2 = nn.Linear(16, 8)   # second hidden layer
        self.fc3 = nn.Linear(8, 1)    # output layer
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return self.fc3(x)  # regression output

# Instantiate model, loss, optimizer
model = DiamondNN()
criterion = nn.MSELoss()          # regression loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Train the network
for epoch in range(1000):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

# Evaluate on test set
y_pred = model(X_test).detach().numpy()
y_true = y_test.numpy()

# Calculate Mean Squared Error
mse = mean_squared_error(y_true, y_pred)
print("Test set MSE:", mse)

# Optionally, show first 10 predictions vs actual
for i in range(10):
    print(f"Predicted: {y_pred[i][0]:.2f}, Actual: {y_true[i][0]:.2f}")


Test set MSE: 2055658.375
Predicted: 463.19, Actual: 559.00
Predicted: 1759.38, Actual: 2201.00
Predicted: 1053.30, Actual: 1238.00
Predicted: 1206.09, Actual: 1304.00
Predicted: 10596.84, Actual: 6901.00
Predicted: 4149.19, Actual: 3011.00
Predicted: 1655.99, Actual: 1765.00
Predicted: 1334.42, Actual: 1679.00
Predicted: 1978.69, Actual: 2102.00
Predicted: 7071.21, Actual: 4789.00
